In [1]:
import pandas as pd

data = {
    "Company": [
        "A","B","A","B","A","B","A","B","A","B",
        "A","B","A","B","A","B","A","B","A","B"
    ],
    "Price_date": [
        "2020-10-01","2020-10-01","2020-10-02","2020-10-02","2020-10-03","2020-10-03",
        "2020-10-04","2020-10-04","2020-10-05","2020-10-05","2020-10-06","2020-10-06",
        "2020-10-07","2020-10-07","2020-10-08","2020-10-08","2020-10-09","2020-10-09",
        "2020-10-10","2020-10-10"
    ],
    "Price": [
        50,89,36,24,39,37,42,63,30,65,
        47,56,71,50,80,54,75,30,63,32
    ]
}

df = pd.DataFrame(data)

# Ensure date column is datetime
df["Price_date"] = pd.to_datetime(df["Price_date"])

print(df)


   Company Price_date  Price
0        A 2020-10-01     50
1        B 2020-10-01     89
2        A 2020-10-02     36
3        B 2020-10-02     24
4        A 2020-10-03     39
5        B 2020-10-03     37
6        A 2020-10-04     42
7        B 2020-10-04     63
8        A 2020-10-05     30
9        B 2020-10-05     65
10       A 2020-10-06     47
11       B 2020-10-06     56
12       A 2020-10-07     71
13       B 2020-10-07     50
14       A 2020-10-08     80
15       B 2020-10-08     54
16       A 2020-10-09     75
17       B 2020-10-09     30
18       A 2020-10-10     63
19       B 2020-10-10     32


In [23]:
import pandas as pd
from src.executor.match_recognize import match_recognize

data = {
    "Company": [
        "A","B","A","B","A","B","A","B","A","B",
        "A","B","A","B","A","B","A","B","A","B"
    ],
    "Price_date": [
        "2020-10-01","2020-10-01","2020-10-02","2020-10-02","2020-10-03","2020-10-03",
        "2020-10-04","2020-10-04","2020-10-05","2020-10-05","2020-10-06","2020-10-06",
        "2020-10-07","2020-10-07","2020-10-08","2020-10-08","2020-10-09","2020-10-09",
        "2020-10-10","2020-10-10"
    ],
    "Price": [
        50,89,36,24,39,37,42,63,30,65,
        47,56,71,50,80,54,75,30,63,32
    ]
}

df = pd.DataFrame(data)

# Ensure date column is datetime
df["Price_date"] = pd.to_datetime(df["Price_date"])



# Test simple query without PREV first
query_simple = """
SELECT *
FROM memory.default.stock_price
MATCH_RECOGNIZE (
  PARTITION BY Company
  ORDER BY Price_date
  MEASURES
    FIRST(A.Price) AS start_price,
    LAST(A.Price)  AS end_price,
    MATCH_NUMBER() AS match_id
  PATTERN (A B C)
  DEFINE
    B AS B.Price > PREV(B.Price),
    C AS C.Price > PREV(C.Price)
);







"""

print("Testing:")
try:
    result = match_recognize(query_simple, df)
    print(result)
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
print()

Testing:
  Company  start_price  end_price  match_id
0       A           36         36         1
1       A           30         30         2
2       B           24         24         1



In [ ]:
# Test simple query without PREV first
query_simple = """
SELECT *
FROM memory.default.stock_price
MATCH_RECOGNIZE (
  PARTITION BY Company
  ORDER BY Price_date
  MEASURES
    FIRST(A.Price) AS start_price,
    LAST(A.Price)  AS end_price,
    MATCH_NUMBER() AS match_id
  PATTERN (A B C)
  DEFINE
    B AS B.Price > PREV(B.Price),
    C AS C.Price > PREV(C.Price)
);
"""
result = match_recognize(query_simple, df)
print(result)

  Company  start_price  end_price  match_id
0       A           36         36         1
1       A           30         30         2
2       B           24         24         1


In [ ]:
SELECT *
FROM memory.default.stock_price
MATCH_RECOGNIZE (
  PARTITION BY company
  ORDER BY price_date
  MEASURES
    FIRST(A.price) AS start_price,
    LAST(A.price)  AS end_price,
    MATCH_NUMBER() AS match_id
  PATTERN (A B C)
  DEFINE
    B AS B.price > PREV(B.price),
    C AS C.price > PREV(C.price)
);


In [21]:
import pandas as pd

df = pd.DataFrame(data).copy()
df["price_date"] = pd.to_datetime(df["price_date"])
df = df.sort_values(["company", "price_date"]).reset_index(drop=True)

out = []
for comp, g in df.groupby("company", sort=False):
    inc = g["price"].diff() > 0
    start_mask = inc.shift(-1) & inc.shift(-2)
    starts = g[start_mask.fillna(False)].copy()

    if starts.empty:
        continue

    # End-of-window (C) is two rows ahead of A
    end_price = g["price"].shift(-2)
    end_date  = g["price_date"].shift(-2)

    starts["start_date"]  = starts["price_date"]
    starts["end_date"]    = end_date.loc[starts.index].values
    starts["start_price"] = starts["price"]
    starts["end_price"]   = end_price.loc[starts.index].values
    starts["company"]     = comp
    starts["match_id"]    = range(1, len(starts) + 1)

    out.append(starts[["company", "start_date", "end_date", "start_price", "end_price", "match_id"]])

result_span = (
    pd.concat(out, ignore_index=True)
    if out else pd.DataFrame(columns=["company","start_date","end_date","start_price","end_price","match_id"])
)
print(result_span)


  company start_date   end_date  start_price  end_price  match_id
0       A 2020-10-02 2020-10-04           36       42.0         1
1       A 2020-10-05 2020-10-07           30       71.0         2
2       A 2020-10-06 2020-10-08           47       80.0         3
3       B 2020-10-02 2020-10-04           24       63.0         1
4       B 2020-10-03 2020-10-05           37       65.0         2


In [26]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# ---------------------------
# Example stock_price dataset
# ---------------------------
data = {
    "Company": [
        "A","B","A","B","A","B","A","B","A","B",
        "A","B","A","B","A","B","A","B","A","B"
    ],
    "Price_date": [
        "2020-10-01","2020-10-01","2020-10-02","2020-10-02","2020-10-03","2020-10-03",
        "2020-10-04","2020-10-04","2020-10-05","2020-10-05","2020-10-06","2020-10-06",
        "2020-10-07","2020-10-07","2020-10-08","2020-10-08","2020-10-09","2020-10-09",
        "2020-10-10","2020-10-10"
    ],
    "Price": [
        50,89,36,24,39,37,42,63,30,65,
        47,56,71,50,80,54,75,30,63,32
    ]
}

df = pd.DataFrame(data)

# Ensure correct types
df["Price_date"] = pd.to_datetime(df["Price_date"])

# ---------------------------
# MATCH_RECOGNIZE SQL string
# ---------------------------
query_simple = """
SELECT *
FROM memory.default.stock_price
MATCH_RECOGNIZE (
  PARTITION BY Company
  ORDER BY Price_date
  MEASURES
    FIRST(A.Price) AS start_price,
    LAST(A.Price)  AS end_price,
    MATCH_NUMBER() AS match_id
  PATTERN (A B C)
  DEFINE
    B AS B.Price > PREV(B.Price),
    C AS C.Price > PREV(C.Price)
);
"""

# ---------------------------
# Run match_recognize engine
# ---------------------------
print("Testing MATCH_RECOGNIZE on stock_price:")
try:
    result = match_recognize(query_simple, df)
    print(result)
except Exception as e:
    print("Error while running MATCH_RECOGNIZE:\n", e)
    import traceback
    traceback.print_exc()


Testing MATCH_RECOGNIZE on stock_price:
  Company  start_price  end_price  match_id
0       A           36         36         1
1       A           30         30         2
2       B           24         24         1


In [ ]:
import pandas as pd

def detect_two_consecutive_increases(
    df: pd.DataFrame,
    company_col: str = "Company",
    date_col: str = "Price_date",
    price_col: str = "Price",
) -> pd.DataFrame:
    
    # Ensure proper dtypes and ordering
    gdf = df.copy()
    gdf[date_col] = pd.to_datetime(gdf[date_col])
    gdf = gdf.sort_values([company_col, date_col]).reset_index(drop=True)

    out_rows = []

    for company, g in gdf.groupby(company_col, sort=False):
        g = g.reset_index(drop=True)
        # inc[k] is True iff price[k] > price[k-1] (within the company partition)
        inc = g[price_col].diff() > 0
        # Candidate starts i for windows (i, i+1, i+2) with two consecutive increases:
        #   price[i+1] > price[i]  and  price[i+2] > price[i+1]
        start_mask = inc.shift(-1).fillna(False) & inc.shift(-2).fillna(False)
        candidates = [int(i) for i, ok in enumerate(start_mask) if ok and i + 2 < len(g)]

        # Greedy selection with AFTER MATCH SKIP PAST LAST ROW (no overlap)
        match_id = 0
        next_start_pos = 0
        for i in candidates:
            if i < next_start_pos:
                continue
            # Accept match (A at i, B at i+1, C at i+2)
            match_id += 1
            out_rows.append({
                "company": company,
                # FIRST(A.price) and LAST(A.price) are the same when A appears once
                "start_price": g.loc[i, price_col],
                "end_price":   g.loc[i, price_col],
                "match_id":    match_id,
            })
            # Skip past the last row of the current match (i+2)
            next_start_pos = i + 3

    return pd.DataFrame(out_rows)

if __name__ == "__main__":
    data = {
        "Company": [
            "A","B","A","B","A","B","A","B","A","B",
            "A","B","A","B","A","B","A","B","A","B"
        ],
        "Price_date": [
            "2020-10-01","2020-10-01","2020-10-02","2020-10-02","2020-10-03","2020-10-03",
            "2020-10-04","2020-10-04","2020-10-05","2020-10-05","2020-10-06","2020-10-06",
            "2020-10-07","2020-10-07","2020-10-08","2020-10-08","2020-10-09","2020-10-09",
            "2020-10-10","2020-10-10"
        ],
        "Price": [
            50,89,36,24,39,37,42,63,30,65,
            47,56,71,50,80,54,75,30,63,32
        ]
    }

    df = pd.DataFrame(data)
    result = detect_two_consecutive_increases(df, "Company", "Price_date", "Price")
    print(result)


  company  start_price  end_price  match_id
0       A           36         36         1
1       A           30         30         2
2       B           24         24         1


/tmp/ipykernel_68022/258436313.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  start_mask = inc.shift(-1).fillna(False) & inc.shift(-2).fillna(False)


In [28]:
import pandas as pd

def detect_two_consecutive_increases(
    df: pd.DataFrame,
    company_col: str = "Company",
    date_col: str = "Price_date",
    price_col: str = "Price",
) -> pd.DataFrame:

    gdf = df.copy()
    gdf[date_col] = pd.to_datetime(gdf[date_col])
    gdf = gdf.sort_values([company_col, date_col]).reset_index(drop=True)

    out_rows = []

    for company, g in gdf.groupby(company_col, sort=False):
        g = g.reset_index(drop=True)
        inc = g[price_col].diff() > 0
        start_mask = inc.shift(-1).fillna(False) & inc.shift(-2).fillna(False)
        candidates = [int(i) for i, ok in enumerate(start_mask) if ok and i + 2 < len(g)]

        match_id = 0
        next_start_pos = 0
        for i in candidates:
            if i < next_start_pos:
                continue
            match_id += 1
            out_rows.append({
                "company": company,
                "start_price": g.loc[i, price_col],
                "end_price":   g.loc[i, price_col],  # matches Trino’s query
                "match_id":    match_id,
            })
            next_start_pos = i + 3  # skip past last row in the match

    return pd.DataFrame(out_rows)


# Run the equivalent of the MATCH_RECOGNIZE query
result = detect_two_consecutive_increases(df)
print(result)


  company  start_price  end_price  match_id
0       A           36         36         1
1       A           30         30         2
2       B           24         24         1


/tmp/ipykernel_68022/3749776361.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  start_mask = inc.shift(-1).fillna(False) & inc.shift(-2).fillna(False)
